## LLM's Political Bias Evaluation using ChatGPT API and Supreme Court opinions.

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [74]:
import json

def compress_jsonl(input_file, output_file):
    with open(input_file, 'r') as infile:
        current_json = ""
        for line in infile:
            current_json += line
            try:
                json_obj = json.loads(current_json)
                with open(output_file, 'a') as outfile:
                    outfile.write(json.dumps(json_obj, separators=(',',':')) + '\n')
                current_json = ""
            except json.JSONDecodeError:
                continue
    
compress_jsonl('courts_for_gpt.jsonl', 'compressed_courts_for_gpt.jsonl')

In [75]:
df = pd.read_json('compressed_courts_for_gpt.jsonl', lines=True)
df

,resource_uri,id,absolute_url,cluster_id,cluster,author_id,author,joined_by,date_created,date_modified,...,plain_text,html,html_lawbox,html_columbia,html_anon_2020,xml_harvard,html_with_citations,extracted_by_ocr,ordering_key,opinions_cited
0,https://www.courtlistener.com/api/rest/v4/opin...,103875,/opinion/103875/hirabayashi-v-united-states/,103875,https://www.courtlistener.com/api/rest/v4/clus...,3115.0,https://www.courtlistener.com/api/rest/v4/peop...,[],2010-04-28T09:54:39-07:00,2022-03-07T14:04:30.030299-08:00,...,,"<p class=""case_cite"">320 U.S. 81</p>\n <p c...",<div>\n<center><b>320 U.S. 81 (1943)</b></cent...,,,,"<div>\n<center><b><span class=""citation no-lin...",False,NaN,[https://www.courtlistener.com/api/rest/v4/opi...
1,https://www.courtlistener.com/api/rest/v4/opin...,9419386,/opinion/103875/hirabayashi-v-united-states/,103875,https://www.courtlistener.com/api/rest/v4/clus...,NaN,None,[],2023-08-02T15:49:12.921336-07:00,2024-09-05T09:19:04.336904-07:00,...,,,,,,"<opinion type=""majority"">\n<author id=""b149-3""...",,False,1.0,[]
2,https://www.courtlistener.com/api/rest/v4/opin...,9419387,/opinion/103875/hirabayashi-v-united-states/,103875,https://www.courtlistener.com/api/rest/v4/clus...,NaN,None,[],2023-08-02T15:49:12.925749-07:00,2024-09-05T09:19:04.343347-07:00,...,,,,,,"<opinion type=""concurrence"">\n<author id=""b171...",,False,2.0,[]
3,https://www.courtlistener.com/api/rest/v4/opin...,9419388,/opinion/103875/hirabayashi-v-united-states/,103875,https://www.courtlistener.com/api/rest/v4/clus...,NaN,None,[],2023-08-02T15:49:12.928835-07:00,2024-09-05T09:19:04.347692-07:00,...,,,,,,"<opinion type=""concurrence"">\n<author id=""b175...",,False,3.0,[]
4,https://www.courtlistener.com/api/rest/v4/opin...,9419389,/opinion/103875/hirabayashi-v-united-states/,103875,https://www.courtlistener.com/api/rest/v4/clus...,NaN,None,[],2023-08-02T15:49:12.931232-07:00,2024-09-05T09:19:04.351779-07:00,...,,,,,,"<opinion type=""concurrence"">\n<author id=""b180...",,False,4.0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,https://www.courtlistener.com/api/rest/v4/opin...,111375,/opinion/111375/wayte-v-united-states/,111375,https://www.courtlistener.com/api/rest/v4/clus...,2611.0,https://www.courtlistener.com/api/rest/v4/peop...,[],2010-04-28T10:13:11-07:00,2020-02-21T14:20:41.851606-08:00,...,,"<p class=""case_cite"">470 U.S. 598</p>\n <p ...",<div>\n<center><b>470 U.S. 598 (1985)</b></cen...,,,,"<div>\n<center><b><span class=""citation no-lin...",False,NaN,[https://www.courtlistener.com/api/rest/v4/opi...
1125,https://www.courtlistener.com/api/rest/v4/opin...,9429952,/opinion/111375/wayte-v-united-states/,111375,https://www.courtlistener.com/api/rest/v4/clus...,NaN,None,[],2023-08-02T16:28:25.101205-07:00,2024-09-05T09:27:34.598379-07:00,...,,,,,,"<opinion type=""majority"">\n<author id=""AFF1"">J...",,False,1.0,[]
1126,https://www.courtlistener.com/api/rest/v4/opin...,9429953,/opinion/111375/wayte-v-united-states/,111375,https://www.courtlistener.com/api/rest/v4/clus...,NaN,None,[],2023-08-02T16:28:25.107688-07:00,2024-09-05T09:27:34.604233-07:00,...,,,,,,"<opinion type=""dissent"">\n<author id=""b670-8"">...",,False,2.0,[]
1127,https://www.courtlistener.com/api/rest/v4/opin...,325463,/opinion/325463/edward-m-kennedy-v-arthur-f-sa...,325463,https://www.courtlistener.com/api/rest/v4/clus...,NaN,None,[],2011-08-23T01:57:09-07:00,2023-08-04T15:16:01.015475-07:00,...,,"<p class=""case_cite"">511 F.2d 430</p>\n <p ...",,,,"<?xml version=""1.0"" encoding=""utf-8""?>\n<opini...","<p class=""case_cite""><span class=""citation no-...",False,NaN,[https://www.courtlistener.com/api/rest/v4/opi...


In [76]:
((df == "") | df.isnull()).sum().sort_values(ascending=False)

html_anon_2020         1129
joined_by_str          1129
html_columbia          1129
page_count              901
plain_text              899
local_path              899
html                    868
author_id               846
author                  846
html_lawbox             812
download_url            766
ordering_key            645
html_with_citations     484
sha1                    484
author_str              394
xml_harvard             385
extracted_by_ocr          0
resource_uri              0
id                        0
type                      0
per_curiam                0
date_modified             0
date_created              0
joined_by                 0
cluster                   0
cluster_id                0
absolute_url              0
opinions_cited            0
dtype: int64

In [77]:
mask = df[['html', 'xml_harvard', 'plain_text', 'html_lawbox']].apply(lambda x: (~x.isnull() & (x != ""))).any(axis=1)

rows_with_at_least_one = df[mask]
rows_with_at_least_one.shape[0] == df.shape[0]

True

In [88]:
import json

def create_batch(custom_id, method, url, messages, output_file):
    batch_entry = {
        "custom_id": custom_id,
        "method": method,
        "url": url,
        "body": {
            "model": "gpt-4o-mini",
            "messages": messages,
            "max_tokens": 9000
        }
    }
    with open(output_file, 'a') as outfile:
        outfile.write(json.dumps(batch_entry, separators=(',',':')) + '\n')

output_file = 'batch_input.jsonl'
method = "POST"
url = "/v1/chat/completions"

for index, row in df.iterrows():
    valid_column = next(
        (col for col in ['html', 'xml_harvard', 'plain_text', 'html_lawbox'] if pd.notnull(row[col]) and row[col] != ""), 
        None
    )

    custom_id = f"request-{index + 1}"
    messages = [
    {
        "role": "system", 
        "content": "Your goal is to determine whether each opinion is in favor of or opposing the President. You will receive the text of a Supreme Court opinion, and you should analyze the sentiment and key phrases to make your determination."
    },
    {
        "role": "user", 
        "content": f"""Here is the text of a Supreme Court opinion: {row[valid_column]}.
        Based on this opinion, please analyze the sentiment and determine if it is in favor of or opposing the President. 
        Return your response in JSON format:
            {{
                "resource": "{row['resource_uri']}",
                "opinion": "with values 'in favor' or 'opposed'",
                "explanation": "a brief explanation of your conclusion"
            }}
        """
    }
]

    create_batch(custom_id, method, url, messages, output_file)



In [89]:
import json

def test_jsonl(input_file, output_file):
    with open(input_file, 'r') as infile:
        line_count = 0 
        for line in infile:
            if line_count >= 25: 
                break
            try:
                json_obj = json.loads(line)
                with open(output_file, 'a') as outfile:
                    outfile.write(json.dumps(json_obj) + '\n')
            except json.JSONDecodeError:
                continue
            line_count += 1  

test_jsonl('batch_input.jsonl', 'test_batch.jsonl')

In [99]:
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
client = OpenAI()

batch_input_file = client.files.create(
  file=open("test_batch.jsonl", "rb"),
  purpose="batch"
)

batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "test job"
    }
)

Batch(id='batch_671651eebef481908276f6239e4644a2', completion_window='24h', created_at=1729516014, endpoint='/v1/chat/completions', input_file_id='file-IzY2R433fuqhUddPKAmqDozL', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1729602414, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'test job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [130]:
client.batches.retrieve("batch_671651eebef481908276f6239e4644a2")

Batch(id='batch_671651eebef481908276f6239e4644a2', completion_window='24h', created_at=1729516014, endpoint='/v1/chat/completions', input_file_id='file-IzY2R433fuqhUddPKAmqDozL', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1729517115, error_file_id=None, errors=None, expired_at=None, expires_at=1729602414, failed_at=None, finalizing_at=1729517113, in_progress_at=1729516015, metadata={'description': 'test job'}, output_file_id='file-oBekUiIdHx5wXfOkt25Jdj3H', request_counts=BatchRequestCounts(completed=25, failed=0, total=25))

In [131]:
import json

batch_info = client.batches.retrieve("batch_671644833c5881908872e8c0f427e423")

input_file_response = client.files.content(batch_info.output_file_id)
print(input_file_response.text)

output_jsonl_file = 'response_test.jsonl' 

for line in input_file_response.text.splitlines():
    try:
        json_obj = json.loads(line)  
        with open(output_jsonl_file, 'a') as outfile:
            outfile.write(json.dumps(json_obj) + '\n')  
    except json.JSONDecodeError:
        continue

def content_jsonl(input_file, output_file):
    with open(input_file, 'r') as infile:
        for line in infile:
            try:
                json_obj = json.loads(line) 
                content = json_obj.get('response', {}).get('body', {}).get('choices', [{}])[0].get('message', {}).get('content', '') 
                with open(output_file, 'a') as outfile:
                    outfile.write(json.dumps(content) + '\n')
            except json.JSONDecodeError:
                continue
content_jsonl('response_test.jsonl', 'content_test.jsonl')

{"id": "batch_req_67164494e39881908f071aac4c98a57c", "custom_id": "request-2258", "response": {"status_code": 200, "request_id": "ff2eed8401f0be74f4728fff6ca6dace", "body": {"id": "chatcmpl-AKlW8YESdD1qiLpfVaK3pAKk0eSB8", "object": "chat.completion", "created": 1729512580, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "```json\n{\n  \"opinion\": \"in favor\",\n  \"explanation\": \"The opinion emphasizes judicial deference to the President's decisions regarding military and national security matters, particularly in the context of the ongoing conflict with terrorism. The court supports the government's authority to detain individuals classified as enemy combatants, indicating that the actions taken by the President in response to the September 11 attacks were appropriate and legally justified.\"\n}\n```", "refusal": null}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 5649, "completion_tokens": 88, "tota

In [125]:
# import pandas as pd

# def retrieve_opinions(input_file):
#     opinions = []
#     explanations = []
    
#     with open(input_file, 'r') as infile:
#         for line in infile:
#             try:
#                 json_obj = json.loads(line)
#                 opinions.append(json_obj.get("opinion"))
#                 explanations.append(json_obj.get("explanation"))
#             except json.JSONDecodeError:
#                 continue
    
#     return pd.DataFrame({'opinion': opinions, 'explanation': explanations})

# output_file = 'output_file.jsonl'
# df_opinions = retrieve_opinions(input_file_response)
# print(df_opinions)